# Creating NDVI Timeseries for Different Landsat Sensors using Google Earth Engine

This notebook outlines how `pixltsnorm` can be used to create timeseries dataframes of NDVI at each pixel for three different landsat sensors using Google Earth Engine.

First, let's import the base packages:

In [1]:
import ee
import fiona
import pandas as pd
import datetime
from tqdm import tqdm

ee.Authenticate()
ee.Initialize()

Now, lets import the pixltsnorm modules:

In [2]:
from pixltsnorm.earth_engine.landsat import cloudMaskL457, scale_factors, addNDVI, create_reduce_region_function

First, let's ' set the dates for each sensor that we want and define the region of interest.

In [3]:
start_date_l5 = '1985-01-01'
end_date_l5 = '2012-04-30'
start_date_l7 = '2000-01-01'
end_date_l7 = '2022-12-31'
start_date_l8 = '2013-04-01'
end_date_l8 = '2022-12-31'

with fiona.open('../example_data/pixel_centers_sm.gpkg') as layer:
    minx, miny, maxx, maxy = layer.bounds
    bbox = ee.Geometry.Rectangle([minx, miny, maxx, maxy])

Now, we can filter each landsat collection, process them to remove clouds, applying scaling factors for collection 2 surface reflectance (see https://developers.google.com/earth-engine/landsat_c1_to_c2 for details), and add an NDVI band. Then, we merge them, and sort by date:

In [4]:
# Landsat 5 collection
collection_l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
    .filterDate(start_date_l5, end_date_l5) \
    .filterBounds(bbox) \
    .map(lambda image: cloudMaskL457(image, 'LANDSAT_5')) \
    .map(lambda image: scale_factors(image)) \
    .map(lambda image: addNDVI(image.set('SPACECRAFT_ID', 'LANDSAT_5')))

# Landsat 7 collection
collection_l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
    .filterDate(start_date_l7, end_date_l7) \
    .filterBounds(bbox) \
    .map(lambda image: cloudMaskL457(image, 'LANDSAT_7')) \
    .map(lambda image: scale_factors(image)) \
    .map(lambda image: addNDVI(image.set('SPACECRAFT_ID', 'LANDSAT_7')))

# Landsat 8 collection
collection_l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate(start_date_l8, end_date_l8) \
    .filterBounds(bbox) \
    .map(lambda image: cloudMaskL457(image, 'LANDSAT_8')) \
    .map(lambda image: scale_factors(image)) \
    .map(lambda image: addNDVI(image.set('SPACECRAFT_ID', 'LANDSAT_8')))

# Merge collections
merged_collection = ee.ImageCollection(collection_l5.merge(collection_l7).merge(collection_l8))

# Sort the collection by date
sorted_collection = merged_collection.sort('system:time_start')

sorted_collection5 = collection_l5.sort('system:time_start')
sorted_collection7 = collection_l7.sort('system:time_start')
sorted_collection8 = collection_l8.sort('system:time_start')

Now we can create the time series data and save it locally. First, we will define a function to process the pixels (converted to points). This code is highly un-optimized and works for small regions but will take a very long time for large regions as it calls `getInfo()` on every loop. If doing this for a larger region, consider refactoring this to run this operation completely server-side on Google Earth Engine.


In [5]:
def process_poi(poi, sorted_collection):
    reduce_to_point = create_reduce_region_function(
        geometry=poi,
        reducer=ee.Reducer.first(),
        scale=30,
        crs='EPSG:4326'
    )
    stat_fc = ee.FeatureCollection(
        sorted_collection.map(
            reduce_to_point
        )
    )
    features = stat_fc.getInfo()['features']
    lon, lat = poi.coordinates().getInfo()
    data = []

    for feature in features:
        properties = feature['properties']
        millis = properties.get('millis')
        date = datetime.datetime.fromtimestamp(millis / 1000).strftime('%Y-%m-%d')
        ndvi = properties.get('NDVI')
        data.append((date, ndvi))

    full_df = pd.DataFrame(data, columns=['Date', 'NDVI'])
    full_df['Date'] = pd.to_datetime(full_df['Date'])
    full_df = full_df.sort_values('Date')

    ndvi_max = full_df.groupby(full_df['Date'].dt.strftime('%Y-%m'))['NDVI'].max().to_dict()

    ndvi_max.update({'lon': lon, 'lat': lat})

    return ndvi_max


Now, apply this to every mission collection:

In [7]:
pois = []
with fiona.open('../example_data/pixel_centers_sm.gpkg') as layer:
    for i, feature in enumerate(layer):
        pnt = feature['geometry']['coordinates']
        pois.append(ee.Geometry.Point(*pnt))

for output, collection in [("landsat5_ndvi.csv", sorted_collection5), ("landsat7_ndvi.csv", sorted_collection7), ("landsat8_ndvi.csv", sorted_collection8)]:
    poi_args = [(poi, collection) for poi in pois]
    results = []
    for arg in tqdm(poi_args, total=len(pois)):
        results.append(process_poi(*arg))

    df_ndvi = pd.DataFrame(list(results))

    df_ndvi.to_csv(output, index=False)

100%|██████████| 130/130 [02:16<00:00,  1.05s/it]


You're done! Now you can harmonize this time series using the `harmonize` or `global_harmonize` module.